# Краулеры

**План**

1. Что такое краулеры?
2. Как написать простой краулер?
3. Блокировки и способы их обхода

## Что такое краулеры?

Краулеры - это боты/программы, которые "ползают" по страницам сайта и собирают информацию. Все чаще использование таких программ запрещается правилами пользования сайтами, поэтому это формально нехорошо. Но так продолжают делать и это надо уметь.   
Запрещают по 2 основным причинам: 
- не хотят делиться данными 
- боятся, что вы уроните сервер (если сайт маленький и сервер не очень, то это довольно легко).    

Поэтому нужно собирать данные аккуратно, чтобы
- вас не заблокировали по IP
- вы не навредили серверу

## Как написать простой краулер?

In [4]:
import requests
from pprint import pprint

Вместо отдельных запросов лучше создать сессию, которая позволит хранить информацию между запросами и поддерживать то же соединение и не создавать каждый раз все заново, что влияет на производительность.

Первая оптимизация, которую стоит принять во внимание при работе с HTTP, заключается в использовании постоянных соединений с веб-серверами. При использовании библиотеки requests в простом режиме (например — применяя её метод get, как в предыдущем ноутбуке) соединение с сервером закрывается после получения ответа от него. 
Для того чтобы этого избежать, приложению нужно использовать объект Session, который позволяет многократно использовать открытые соединения:

In [7]:
session = requests.session() # Объект session будет хранить открытые соединения с вэб-серверами

Соединения хранятся в пуле соединений (он, по умолчанию, рассчитан на 10 соединений). Размер пула можно настраивать:

In [8]:
adapter = requests.adapters.HTTPAdapter(
    pool_connections=10, # Количество пулов соединений
    pool_maxsize=10 # Размер каждого пула
)

In [9]:
session.mount('https://', adapter)

Попробуем сделать запрос, просто вместо requests.get мы пишет session.get.

In [10]:
response = session.get('https://ru.wikipedia.org')

Может найти значения нашего IP-адреса

In [11]:
response.headers['X-Client-IP']

'81.200.8.153'

Можно ли нас отследить по IP? С определенной точностью, можно узнать округ или компанию, к которой привязан любой IP. Пример сервиса, который позволяет это сделать [здесь](https://whatismyipaddress.com/ip-lookup)

Посмотреть на headers запроса

In [15]:
pprint(dict(response.headers))

{'accept-ch': 'Sec-CH-UA-Arch,Sec-CH-UA-Bitness,Sec-CH-UA-Full-Version-List,Sec-CH-UA-Model,Sec-CH-UA-Platform-Version',
 'accept-ranges': 'bytes',
 'age': '2793',
 'cache-control': 'private, s-maxage=0, max-age=0, must-revalidate',
 'content-encoding': 'gzip',
 'content-language': 'ru',
 'content-length': '26581',
 'content-type': 'text/html; charset=UTF-8',
 'date': 'Fri, 18 Nov 2022 20:00:31 GMT',
 'last-modified': 'Fri, 18 Nov 2022 20:00:28 GMT',
 'nel': '{ "report_to": "wm_nel", "max_age": 86400, "failure_fraction": 0.05, '
        '"success_fraction": 0.0}',
 'permissions-policy': 'interest-cohort=(),ch-ua-arch=(self '
                       '"intake-analytics.wikimedia.org"),ch-ua-bitness=(self '
                       '"intake-analytics.wikimedia.org"),ch-ua-full-version-list=(self '
                       '"intake-analytics.wikimedia.org"),ch-ua-model=(self '
                       '"intake-analytics.wikimedia.org"),ch-ua-platform-version=(self '
                       '"intak

### Стратегии сбора данных


По сути краулеры выполняют сбор страниц (их html) как мы это делали на прошлом занятии, но делают они это циклами (или циклами циклов). Можно выделить разные стратегии сбора данных:
    
**По типу навигации**

1. Все страницы со ссылками имеют удобные номера ("https://ficbook.net/fanfiction/no_fandom/originals?p=2"), обычно просто p=(число) или page=(число). В этом случае вам нужно просто подставлять цифры подробнее про параметры передаваемые в ссылке можно посмотреть [здесь](https://en.wikipedia.org/wiki/Query_string)
2. Страницы называются как-то не структурированно (например, по названиям блоков). Тут нужно собирать ссылки на эти страницы и потом по ним ходить и собирать конечные странички.
3. Все расположено на одной страничке и догружается с использованием [WebSocket](https://en.wikipedia.org/wiki/WebSocket) или других технологий, при этом адрес в адресной строке никак не изменяется, данные могут догружаться на сайт автоматически по мере скролла страницы

**По скорости обновления**

1. Если сайт довольно статичный по контенту (медленно появляются и удаляются материалы), то можно сначал собрать ссылки, а потом по ним ходить
2. Если сайт очень динамичный по контенту (например, объявления на крупном сайте), вам нужно при получении страничкии ссылок сразу их обходить, а потом переходить к следующей, потому что ко времени получения исчерпывающего списка ссылок по сайту многие будут уже удалены или недоступны



## Блокировки и способы их обхода

Для того, чтобы предотвратить автоматический сбор информации с некого сайта, применяются различные инструменты, которые определяют роботов и блокируют запросы с адресов, которые были классифицированы как роботы. Чтобы не заблокировали домашний/учебный ip, лучше сразу задуматься об этих мерах и предотвратить возможные проблемы. Кстати, Википедия не блокирует и можно спокойно скачивать без каких-либо проблем.

Чтобы их обойти, можно попробовать несколько инструментов:
1. time.sleep(x) - задержка между запросами, чтобы слишком большая скорость запросов не показалась подозрительной или ваши запросы не уронили сервер небольшого ресурса (например, региональной газеты)
2. time.sleep(случайный промежуток времени) - это более хитрая версия, когда время задержки - это случайное число из некоторого отрезка (модуль random)
3. изобразить браузер - при запросе отправляется информация о том, из какого приложения пришел запрос (например, Googlr Chrome), запросы сделанные из браузера больше похожи на человеческие, для этого нужно задать user-agent в параметрах (а его выбирать случайно с помощью fake_useragent)
4. использовать прокси - существуют ресурсы с бесплатными списками открытых прокси, через которые можно пропускать ваш запрос и сервер будет думать, что запросы приходят из разных мест (anonymous и elite классы прокси) или использовать анонимизированные сети к примеру сеть [Tor](https://en.wikipedia.org/wiki/Tor_(network)) и аналоги.

### Пауза между запросами

In [16]:
import time
from datetime import datetime

In [17]:
for _ in range(5):
    response = session.get('https://ru.wikipedia.org')
    print(datetime.now())
    time.sleep(1)

2022-11-18 23:52:41.859211
2022-11-18 23:52:43.104344
2022-11-18 23:52:44.227002
2022-11-18 23:52:45.619185
2022-11-18 23:52:46.745426


### Притвориться нормальным браузером

При отправке запроса на сервер/сайт в параметрах нашего запроса по умолчанию содержится информация кто обращается к серверу.
Если мы никак эту информацию не редактируем, то по умолчанию запрос честно передает серверу информацию что запрос идет из Python,
библиотеки Requests.

Но многие сайты не желают обрабатывать такие запросы, так как информацию запрашивает не человек сидя в браузере, а какая-то программа.
Поэтому они иногда ставят блокировки и фильтры на входящие запросы, так они отсеивают всяких ботов.  

Но в Python есть возможность прикинуться браузером. Можно вручную задать устройство/браузер, можно использовать специальную библиотеку. Разберем оба случая.

In [19]:
from fake_useragent import UserAgent

Можно настроить так, чтобы не проверять безопасность соединения, что иногда вызывает ошибки. Но это можно делать с сайтмаи, которым вы доверяете.

In [20]:
ua = UserAgent(verify_ssl=False)

In [22]:
ua.random

'Mozilla/5.0 (Windows; U; Windows NT 5.1; ja-JP) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.3 Safari/533.19.4'

In [23]:
ua.random

'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:77.0) Gecko/20190101 Firefox/77.0'

In [24]:
headers = {'User-Agent': ua.random}
print(headers)
response = session.get('https://ru.wikipedia.org', headers=headers)

{'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 6.1; tr-TR) AppleWebKit/533.20.25 (KHTML, like Gecko) Version/5.0.4 Safari/533.20.27'}


### Пауза между запросами (случайное время)

In [25]:
import random

random.uniform позволяет получить случайное число из отрезка

In [26]:
random.uniform(1, 3)

1.0659317490373563

In [27]:
for _ in range(5):
    response = session.get('https://ru.wikipedia.org')
    print(datetime.now())
    time.sleep(random.uniform(1.1, 5.2))

2022-11-18 23:56:17.413681
2022-11-18 23:56:22.143512
2022-11-18 23:56:25.033615
2022-11-18 23:56:27.320069
2022-11-18 23:56:30.951056


### Подключение через прокси

Прокси-сервер — это дополнительное звено между вами и интернетом, через него пойдет подключение и сайт не будет знать, что это вы посылаете запрос.

Адреса прокси можно взять со специальных сайтов, например, [https://hideip.me/ru/proxy/httplist](https://hideip.me/ru/proxy/httplist). И потом проверить, что они рабочие, прежде чем использовать [https://checkerproxy.net/](https://checkerproxy.net/)

In [34]:
known_proxy_ip = '167.235.6.102:10061'
known_proxy_ip = '3.28.199.73:8080'
known_proxy_ip = '128.199.77.142:5001'
proxy = {'http': known_proxy_ip, 'https': known_proxy_ip}
response = session.get('http://ru.wikipedia.org', proxies=proxy)
# print(response.headers['X-Client-IP'])

In [35]:
response.text

'<!DOCTYPE html>\n<html class="client-nojs" lang="ru" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Википедия\xa0— свободная энциклопедия</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\\t.","\xa0\\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","январь","февраль","март","апрель","май","июнь","июль","август","сентябрь","октябрь","ноябрь","декабрь"],"wgRequestId":"1bc162ca-8e53-481b-b815-c298d013af81","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Заглавная_страница","wgTitle":"Заглавная страница","wgCurRevisionId":121103235,"wgRevisionId":121103235,"wgArticleId":4401,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Википедия:Страницы с ежечасно очищаемым кэшем","Персоналии по алфавиту","Родившиеся 21 сентября",\n"Родившиеся в 1407 году"

## Примеры

### Пример 1

Давайте обкачаем немного новостей с сайта вышки.

1. Страницы имеют вид "https://www.hse.ru/news/page1.html", поэтому можно просто идти циклом.
2. Достанем дату публикации, заголовок, краткое описание (из станицы со списком новостей), текст полной статьи и метки (из самой страницы новости)
3. Положим в базу

In [36]:
import sqlite3
from bs4 import BeautifulSoup
import re

Давайте зайдем в DB Browser for SQLite и создадим базу данных, куда будем сохранять страницы, которые спарсили.

In [138]:
conn = sqlite3.connect('vk_profiles.db')
cur = conn.cursor()

Создаем базу данных, где будем хранить информацию

In [139]:
cur.execute("""
CREATE TABLE IF NOT EXISTS vk_profiles 
(id INTEGER PRIMARY KEY, username text, city text, birthday text, subscribers int, posts int, photos int)
""")
conn.commit()
conn.close()

### Отработаем процесс на одной странице

**Шаг 1. Найти страницы**

Посмотрим, как устроены новости и скачаем одну страницу

In [116]:
req = session.get('https://vk.com/id100', headers={'User-Agent': ua.random})

Распарсим с помощью BeautifulSoup

In [63]:
soup = BeautifulSoup(req.text, 'html.parser')

In [95]:
profile_block = soup.find('div', {'class': 'page_info_wrap'}) # Вытащим блок HTML отвечающий за общие данные по человеку

In [97]:
profile_block.find('h1', {'class': 'page_name'}).text.strip() # Найдем в нем название страницы/юзера

'Лиза  Щепилова (Богданова)'

In [86]:
# Теперь попробуем вытащить информацию по дате рождения и родному городу
profile_rows = profile_block.find_all('div', {'class': 'clear_fix profile_info_row'})
profile_rows

[<div class="clear_fix profile_info_row">
 <h3 class="label fl_l">День рождения:</h3>
 <div class="labeled"><a href="/search?c[section]=people&amp;c[bday]=23&amp;c[bmonth]=12">23 декабря</a></div>
 </div>,
 <div class="clear_fix profile_info_row">
 <h3 class="label fl_l">Город:</h3>
 <div class="labeled"><a href="/search?c[name]=0&amp;c[section]=people&amp;c[country]=1&amp;c[city]=161">Чита</a></div>
 </div>,
 <div class="clear_fix profile_info_row">
 <h3 class="label fl_l">Семейное положение:</h3>
 <div class="labeled">замужем за <a class="mem_link" href="/dusya_felcone" mention_id="id371667001" onclick="return mentionClick(this, event)" onmouseover="mentionOver(this)">Данилом Щепиловым</a></div>
 </div>,
 <div class="clear_fix profile_info_row">
 <h3 class="label fl_l">Языки:</h3>
 <div class="labeled"><a href="/search?c[name]=0&amp;c[section]=people&amp;c[lang]=-1">Русский</a></div>
 </div>,
 <div class="clear_fix profile_info_row">
 <h3 class="label fl_l">Братья, сёстры:</h3>
 <div

In [76]:
profile_rows[0].text

'\nДень рождения:\n23 декабря\n'

In [77]:
profile_rows[1].text

'\nГород:\nЧита\n'

In [87]:
profile_block.find_all('a', {'class': 'page_counter'})

[]

In [98]:
counts_module = soup.find('div', {'class': 'counts_module'})

In [99]:
counts_module

<div class="counts_module"><a class="page_counter" onclick="return Page.showLoginBox()">
<div class="count">857</div>
<div class="label">подписчиков</div>
</a><a class="page_counter" onclick="return Page.showLoginBox()">
<div class="count">1</div>
<div class="label">запись</div>
</a><a class="page_counter" href="/albums408166137?profile=1" onclick="return Page.showLoginBox()">
<div class="count">129</div>
<div class="label">фотографий</div>
</a><a class="page_counter" href="/tag408166137" onclick="return Page.showLoginBox()">
<div class="count">2</div>
<div class="label">отметки</div>
</a></div>

In [100]:
counts_module.find_all('div', {'class': "count"})

[<div class="count">857</div>,
 <div class="count">1</div>,
 <div class="count">129</div>,
 <div class="count">2</div>]

In [103]:
counts_module.find_all('div', {'class': "label"})

[<div class="label">подписчиков</div>,
 <div class="label">запись</div>,
 <div class="label">фотографий</div>,
 <div class="label">отметки</div>]

**Оформляем нормально в функцию**

In [126]:
def get_vkprofile_stats(uid):
    req = session.get(f'https://vk.com/id{uid}', headers={'User-Agent': ua.random})
    user_stats = dict.fromkeys(['User_Name', 'City', 'Birthday', 'Subscribers', 'Posts', 'Photos'])
    
    soup = BeautifulSoup(req.text, 'html.parser')
    profile_block = soup.find('div', {'class': 'page_info_wrap'})
    if profile_block is not None:
        pgname = profile_block.find('h1', {'class': 'page_name'})
        if pgname:
            user_stats['User_Name'] = pgname.text.strip()

        profile_rows = profile_block.find_all('div', {'class': 'clear_fix profile_info_row'})
        for row in profile_rows:
            if 'Город' in row.text:
                user_stats['City'] = row.text.strip().split('\n')[-1]
            elif 'День рождения' in row.text:
                user_stats['Birthday'] = row.text.strip().split('\n')[-1] 
    
    
    counts_module = soup.find('div', {'class': 'counts_module'})
    if counts_module:
        cnts = counts_module.find_all('div', {'class': "count"})
        lbls = counts_module.find_all('div', {'class': "label"})

        for lb, cnt in zip(lbls, cnts):
            if lb.text == 'подписчиков':
                user_stats['Subscribers'] = int(cnt.text)
            elif lb.text == 'запись':
                user_stats['Posts'] = int(cnt.text)
            elif lb.text == 'фотографий':
                user_stats['Photos'] = int(cnt.text)
    return user_stats

**Пробуем выкачать рандомные страницы**

In [141]:
uids = [np.random.randint(low=400000000, high=500000000) for x in range(20)]

In [143]:
from tqdm import tqdm

In [144]:
collected_data = []
for uid in tqdm(uids):
    collected_data.append((uid, get_vkprofile_stats(uid)))
    time.sleep(random.uniform(1.1, 5.2))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [01:09<00:00,  3.46s/it]


In [145]:
collected_data

[(454158927,
  {'User_Name': 'Арина  Родионова',
   'City': None,
   'Birthday': '12 июня 1997 г.',
   'Subscribers': None,
   'Posts': 1,
   'Photos': None}),
 (465761542,
  {'User_Name': 'Жаксыбек  Сагидулла',
   'City': None,
   'Birthday': '8 августа 2003 г.',
   'Subscribers': None,
   'Posts': None,
   'Photos': None}),
 (431989802,
  {'User_Name': 'Вера  Решетняк-Заржицкая',
   'City': None,
   'Birthday': None,
   'Subscribers': None,
   'Posts': None,
   'Photos': None}),
 (459808168,
  {'User_Name': None,
   'City': None,
   'Birthday': None,
   'Subscribers': None,
   'Posts': None,
   'Photos': None}),
 (470842990,
  {'User_Name': 'Шамиль  Оздоев',
   'City': 'Москва',
   'Birthday': '22 января 2003 г.',
   'Subscribers': None,
   'Posts': 1,
   'Photos': None}),
 (490608931,
  {'User_Name': 'Jig-Jeremiah  Geromo',
   'City': 'Zamboanga',
   'Birthday': '3 декабря 2004 г.',
   'Subscribers': None,
   'Posts': None,
   'Photos': None}),
 (464719195,
  {'User_Name': 'Shahrin 

**Шаг 4. Пишем в базу**

Надо завести словарь для тегов (сначала читаем из базы, а потом дозаписываем), множество виденных статей (чтобы при перезаупске не дублировать)

In [ ]:
ordered_cols = ['User_Name', 'City', 'Birthday', 'Subscribers', 'Posts', 'Photos']
cur.execute(
    """
    INSERT INTO vk_profiles 
        (id, username, city, birthday, subscribers, posts, photos)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, [[row[0], ] + [row[x] for x in ordered_cols]  for row in collected_data]
)

In [149]:
def write_to_db(data):
    # сохраняем информацию по текстам
    ordered_cols = ['User_Name', 'City', 'Birthday', 'Subscribers', 'Posts', 'Photos']
    data = [row for row in data if row[1]['User_Name'] is not None]
    cur.executemany(
        """
        INSERT INTO vk_profiles 
            (id, username, city, birthday, subscribers, posts, photos)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        """, [[row[0], ] + [row[1][x] for x in ordered_cols]  for row in data]
    )
    conn.commit()

In [151]:
conn = sqlite3.connect('vk_profiles.db')
cur = conn.cursor()

In [152]:
write_to_db(collected_data)